In [ ]:
from pathlib import Path
import pandas as pd
import random
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()
import matplotlib.colors as mcolors
from matplotlib.lines import Line2D
import numpy as np
from scipy.optimize import curve_fit

### Compute convergence minimas.

In [ ]:
best_scores = list()
best_accs = list()
for i in range(len(xray_stat_dfs)):
    best_scores.append(xray_stat_dfs[i].iloc[-1]["{}_mean".format(score_field)])
    best_accs.append(rmsd_stat_dfs[i].iloc[-1]["{}_mean".format(rmsd_field)])

print(np.mean(best_scores), np.std(best_scores))
print(np.mean(best_accs), np.std(best_accs))

### Compute convergence points.

In [ ]:
# Define the model function for power law decay with an additional c parameter
def power_law_decay(x, a, b, c):
    return a * np.power(x, b) + c

# Define a function to perform the curve fitting for the power law decay model
def fit_power_law_decay(xs, ys):
    # Make an initial guess for the parameters
    # a: scale factor, b: exponent, c: y-axis shift
    initial_guess = [np.max(ys), -1.0, np.min(ys)]

    # Perform the curve fitting
    params, covariance = curve_fit(power_law_decay, xs, ys, p0=initial_guess, maxfev=10000)

    # Extract the fitted parameters
    a, b, c = params

    return a, b, c

In [ ]:
xray_stat_df = xray_stat_dfs[1]
a, b, c = fit_power_law_decay(xray_stat_df.index, xray_stat_df["{}_mean".format(score_field)])
plt.plot(xray_stat_df.index, xray_stat_df["{}_mean".format(score_field)], label="X-ray Score")
plt.plot(xray_stat_df.index, power_law_decay(xray_stat_df.index, a, b, c), label="Fitted Exponential Curve")

In [ ]:

def find_derivative_zero_point_power_law(a, b, c, sigma, x_start=1.0):
    """
    Find the x value where the derivative of the power law decay function is approximately zero within a sigma value.

    Parameters:
    a (float): The scale factor in the power law equation.
    b (float): The exponent in the power law equation.
    c (float): The constant offset in the power law equation.
    sigma (float): The sigma value within which the derivative is considered approximately zero.
    x_start (float): The starting x value for finding the zero derivative point.

    Returns:
    x_zero_derivative (float): The x value where the first derivative is approximately zero within sigma.
    """

    # Define the derivative of the power law function
    def derivative_power_law(x):
        return a * b * np.power(x, b - 1)

    # Starting point for the search
    x_guess = x_start

    # Define the step size and precision for the search
    step_size = 0.1
    precision = sigma / 10  # Stop when we are within 10% of the sigma value

    # Search for the point where the derivative is approximately zero
    while abs(derivative_power_law(x_guess)) > sigma:
        # Update the guess based on the derivative
        x_guess -= step_size * np.sign(derivative_power_law(x_guess))

        # If the derivative changes sign, make the step size smaller
        if np.sign(derivative_power_law(x_guess - step_size)) != np.sign(derivative_power_law(x_guess)):
            step_size /= 10

        # If the step size is too small, break the loop
        if step_size < precision:
            break

    if abs(derivative_power_law(x_guess)) <= sigma:
        x_zero_derivative = x_guess
    else:
        x_zero_derivative = None  # Could not find a point where the derivative is approximately zero within sigma

    return x_zero_derivative


In [ ]:
score_conv_pts = list()
acc_conv_pts = list()

for i in range(len(xray_stat_dfs)):
    xray_stat_df = xray_stat_dfs[i]
    rmsd_stat_df = rmsd_stat_dfs[i]

    a_score, b_score, c_score = fit_power_law_decay(xray_stat_df.index, xray_stat_df["{}_mean".format(score_field)])
    x_score = find_derivative_zero_point_power_law(a_score, b_score, c_score, sigma=1e-4)

    a_acc, b_acc, c_acc = fit_power_law_decay(rmsd_stat_df.index, rmsd_stat_df["{}_mean".format(rmsd_field)])
    x_acc = find_derivative_zero_point_power_law(a_acc, b_acc, c_acc, sigma=1e-4)

    score_conv_pts.append(x_score)
    acc_conv_pts.append(x_acc)

print(score_conv_pts)
print(np.mean(score_conv_pts), np.std(score_conv_pts))
print(acc_conv_pts)
print(np.mean(acc_conv_pts), np.std(acc_conv_pts))